# Tuning and deploy a foundation model


**Learning Objective**

1. Learn how to generate a JSONL file for Gemini tuning
1. Learn how to launch a tuning job
1. Learn how to deploy and query a tuned LLM
1. Learn how to evaluate a tuned LLM


Creating an LLM requires massive amounts of data, significant computing resources, and specialized skills. In this notebook, you'll learn how tuning allows you to customize a Gemini foundation model on Vertex Generative AI studio for more specific tasks or knowledge domains.
While the prompt design is excellent for quick experimentation, if training data is available, you can achieve higher quality by tuning the model. Tuning a model enables you to customize the model response based on examples of the task you want the model to perform.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

## Setup

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [34]:
import json
import time

from IPython.display import Markdown
import evaluate
import pandas as pd
from google.cloud import aiplatform, bigquery
from sklearn.model_selection import train_test_split
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.tuning import sft

In [3]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [4]:
!gsutil ls $BUCKET_URI || gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

gs://dherin-dev/cls_datasheets.csv
gs://dherin-dev/cord19_embeddings.json
gs://dherin-dev/data.csv
gs://dherin-dev/salads.csv
gs://dherin-dev/tune_data_stack_overflow_python_qa.jsonl
gs://dherin-dev/115851500182/
gs://dherin-dev/7737964263322419200-616112577574862848/
gs://dherin-dev/autosxs-1713380879483659/
gs://dherin-dev/autosxs-1713383655691384/
gs://dherin-dev/autosxs-1713384745904386/
gs://dherin-dev/autosxs-human-eval-1713385691277474/
gs://dherin-dev/autosxs-p8fk0gjz/
gs://dherin-dev/babyweight/
gs://dherin-dev/babyweight_220707_021136/
gs://dherin-dev/babyweight_220707_021151/
gs://dherin-dev/babyweight_220707_021154/
gs://dherin-dev/car_damage_lab_images/
gs://dherin-dev/classification-bert-20230411003650/
gs://dherin-dev/contextual_bandit_checkpoints/
gs://dherin-dev/covertype/
gs://dherin-dev/flowers/
gs://dherin-dev/models/
gs://dherin-dev/movies/
gs://dherin-dev/staging/
gs://dherin-dev/taxifare-20230710171207/
gs://dherin-dev/taxifare-20230710191151/
gs://dherin-dev/tax

## Training Data


In this notebook, we will be tuning Gemini using the Python SDK on a questions & answers dataset from StackOverflow. 
Our first step will be to query the StackOverflow data on BigQuery Public Datasets, limiting to questions with the `python` tag, and `accepted` answers from 2020-01-01 only. 

We will limit the dataset to 1000 samples, 800 of which will be used to tune the LLM and the rest for evaluating the tuned model.
The second step will be to convert the dataset into a JSONL format, with one example per line, so that the tuning job can consume it.


Next let us run the query to assemble our dataset into the DataFrame `df`:

In [5]:
%%bigquery df

SELECT CONCAT(q.title, q.body) as input_text, a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    1000

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
df.head()

,input_text,output_text
0,How to generate coordinate list from matplotli...,"<p>Use <a href=""https://numpy.org/doc/stable/r..."
1,Write files df in loop<p>I have script that ru...,<p>Here is possible simplify your task:</p>\n<...
2,HTML Plotly Dash<p>I found one HTML template t...,"<p>You'll need to add IDs to some components, ..."
3,Converting struct format string to range of al...,<p>The math can be simplified. If <code>b</co...
4,I want to understand which line of code output...,<p>This line:</p>\n<pre><code>print(double_lyr...


The column `input_text` corresponds to the actual questions asked by the StackOverflow users, while the `output_text` column corresponds to the correct answers. From this dataset of 1000 questions-answers pairs, we will now need to generate a JSONL file with one example per line in the format:

```python
{
      "messages": [
        {
          "role": "user",
          "content": input_text,
        },
        {
          "role": "model",
          "content": output_text,
        }
      ]
}
```

This is the format we need to tune a Gemini model.

To tune Gemini we advise at least 100 to 500 examples. The more examples you provide in your dataset, the better the results. There is no limit for the number of examples in a training dataset. In this case you will use 800.
If possible, also provide a validation dataset. A validation dataset helps you measure the effectiveness of a tuning job. Validation datasets support up to 256 examples.

Let's first split the data into training and evaluation. 

In [ ]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
print("train size:", len(train))
print("eval size:", len(evaluation))

In [9]:
train.head()

,input_text,output_text
261,Unable remove row in proper order from qtablev...,<p>Changing the size and layout of the model s...
887,Display numbered list<p>My last question was s...,"<p>this could be like</p>\n<pre class=""lang-py..."
401,Calling functions in a django app from url<p>C...,<p>I think you want something like Subdomains!...
733,install local package in tox unencumbered by p...,"<p>If you want to start from a clean state, yo..."
120,BranchOperator is getting skipped airflow<p>I ...,<p>I cannot reproduce failure in task <code>br...


For tuning, the training and evaluation data first needs to be converted into a JSONL format. For this, we provide the following two helper functions.
The first one converts a single `input_text` and `output_text` record into the JSONL format required by Gemini.

In [10]:
def format_for_gemini(input_text, output_text):
    return json.dumps({
      "messages": [
        {
          "role": "user",
          "content": input_text,
        },
        {
          "role": "model",
          "content": output_text,
        }
      ]
    })


In [ ]:
The second helper function exports the data into a file:

In [ ]:
def export_tuning_data(file_name, df):
    with open(file_name, "a") as file:
        for row in df.iterrows():
            jsonline = format_for_gemini(
                row[1]["input_text"], 
                row[1]["output_text"],
            )
            file.write(jsonline)
            file.write("\n")

Let us now create our training and evaluation files:

In [24]:
training_data_filename = "tune_data_stack_overflow_python_qa.jsonl"
evaluation_data_filename = "evaluation_data_stack_overflow_python_qa.jsonl"

!test -f $training_data_filename    && rm $training_data_filename
!test -f $evaluation_data_filename  && rm $evaluation_data_filename

export_tuning_data(training_data_filename, train)
export_tuning_data(evaluation_data_filename, evaluation)

You can then export the local files to GCS, so that they can be used by Vertex AI for the tuning job.

In [27]:
!gsutil cp $training_data_filename   $BUCKET_URI
!gsutil cp $evaluation_data_filename $BUCKET_URI

Copying file://tune_data_stack_overflow_python_qa.jsonl [Content-Type=application/octet-stream]...
/ [1 files][  2.3 MiB/  2.3 MiB]                                                
Operation completed over 1 objects/2.3 MiB.                                      
Copying file://evaluation_data_stack_overflow_python_qa.jsonl [Content-Type=application/octet-stream]...
/ [1 files][621.7 KiB/621.7 KiB]                                                
Operation completed over 1 objects/621.7 KiB.                                    


You can check to make sure that the files successfully transferred to your Google Cloud Storage bucket:

In [28]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"
EVALUATION_DATA_URI = f"{BUCKET_URI}/{evaluation_data_filename}"

!gsutil ls -al $TRAINING_DATA_URI
!gsutil ls -al $EVALUATION_DATA_URI

   2439571  2024-08-07T01:32:45Z  gs://dherin-dev/tune_data_stack_overflow_python_qa.jsonl#1722994365137329  metageneration=1
TOTAL: 1 objects, 2439571 bytes (2.33 MiB)
    636641  2024-08-07T01:32:47Z  gs://dherin-dev/evaluation_data_stack_overflow_python_qa.jsonl#1722994367446931  metageneration=1
TOTAL: 1 objects, 636641 bytes (621.72 KiB)


### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.


In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

sft_tuning_job = sft.train(
    source_model="gemini-1.0-pro-002",
    train_dataset=TRAINING_DATA_URI,
    # The following parameters are optional
    validation_dataset=EVALUATION_DATA_URI,
    epochs=4,
    adapter_size=4,
    learning_rate_multiplier=1.0,
    tuned_model_display_name="stackoverflow_tuned_gemini_pro",
)

# Polling for job completion
while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(sft_tuning_job.tuned_model_name)
print(sft_tuning_job.tuned_model_endpoint_name)
print(sft_tuning_job.experiment)

Creating SupervisedTuningJob
SupervisedTuningJob created. Resource name: projects/115851500182/locations/us-central1/tuningJobs/5004357251754885120
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/115851500182/locations/us-central1/tuningJobs/5004357251754885120')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/5004357251754885120?project=115851500182


## Retrieve the tuned model from your Vertex AI Model registry


When your tuning job is finished, your model will be available on Vertex. The next cell shows you how to list tuned models.

In [32]:
responses = sft.SupervisedTuningJob.list()

for response in responses:
    print(response)

resource name: projects/115851500182/locations/us-central1/tuningJobs/5004357251754885120


It's time to get predictions. First you need to get retrieve the tune model:

In [35]:
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)

Now you can start sending a prompt to the API. Feel free to update the following prompt:

In [38]:
PROMPT = """
How can I store my TensorFlow checkpoint on Google Cloud Storage?

Python example:

"""

response = tuned_model.generate_content(PROMPT)
Markdown(response.text)

I've used Cloud Storage buckets to store data for my TensorFlow jobs running in Google Cloud AI Platform. I think I can help.

I think your current code is using the `tf.train` saver. To save the checkpoint in Cloud Storage, you will need to use the `tf.train.Checkpoint` API instead.

Here's a snippet:

```python
import os
import tensorflow as tf
from tensorflow.python.lib.io import file_io

# Set up the bucket name and directory
bucket_name = "your_bucket_name"
checkpoint_dir = "your_checkpoint_dir"

# Set up the checkpoint
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

# Set up the checkpoint manager
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=os.path.join(file_io.get_hdfs_root(), bucket_name, checkpoint_dir), max_to_keep=3)

# Save the checkpoint
checkpoint_manager.save()
```

This will save the checkpoint to the specified bucket and directory.

**Note:**

* You will need to create the bucket and directory in Cloud Storage first.
* You will need to set environment variables `GOOGLE_APPLICATION_CREDENTIALS` and `TF_CONFIG` before running your script. The `GOOGLE_APPLICATION_CREDENTIALS` variable should point to a service account key file with the appropriate permissions to access the bucket. The `TF_CONFIG` variable should specify the Cloud AI Platform job name, which you can find in the Cloud Console.

Here's an example of how to set the environment variables:

```bash
export GOOGLE_APPLICATION_CREDENTIALS=/path/to/your/service_account.json
export TF_CONFIG='{"job_name": "my_job_name"}'
```

For more information on using Cloud AI Platform, please see the [documentation](https://cloud.google.com/ai-platform/training/docs/estimator/using-gpus).

I hope this helps!

## Manual Evaluation

It's essential to evaluate your model to understand its performance. Evaluation can be done in an automated way using evaluation metrics like F1, Bleu, or Rouge. You can also leverage human evaluation methods. Human evaluation methods involve asking humans to rate the quality of the LLM's answers. This can be done through crowdsourcing or by having experts evaluate the responses. Some standard human evaluation metrics include fluency, coherence, relevance, and informativeness. Often you want to choose a mix of evaluation metrics to get a good understanding of your model performance. 


Among other metrics we will compute the following two metrics that provide crude measures albeit automated of how two texts may have the same meaning: 
- The [BLEU](https://en.wikipedia.org/wiki/BLEU) evaluation metric is a sort of **precision** metric, measuring the proportion of $n$-grams in the generated sentence matching $n$-grams in the reference sentence. It goes from 0 to 1 with a higher score for more similar sentences. BLEU1 considers uni-grams only, while BLEU2 considers bi-grams. 

- The [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) evaluation metric is a sort of **recall** metric, measuring the proportion of $n$-grams in the reference sentence that are matched by $n$-grams in the generated sentence. It goes from 0 to 1 with a higher score for more similar sentences. ROUGE1 considers uni-grams only, while ROUGE2 considers bi-grams.


We will use  [evaluate](https://github.com/huggingface/evaluate/tree/main) to to compute the scores.
Earlier in the notebook, you created a train and eval dataset. Now it's time to take some of the eval data. You will use the questions to get a response from our tuned model, and the answers we will use as a reference:
- **Candidates**: Answers generated by the tuned model.
- **References**: Original answers that we will use to compare


In [39]:
# you can change the number of rows you want to use
EVAL_ROWS = 60
INPUT_LIMIT = 10000  # characters
evaluation = evaluation[evaluation.input_text.apply(len) <= INPUT_LIMIT]
evaluation = evaluation.head(EVAL_ROWS)
evaluation.head()

,input_text,output_text
771,"How can I join a df made in iteration, to buil...","<pre><code>&gt;&gt;&gt; col_names = ['Code', '..."
56,Azure Machine Learning Python Module failing t...,"<p>After speaking with Microsoft Support, it t..."
440,faster way to fetch the data from postgresql i...,<p>For this query:</p>\n<pre><code>SELECT scri...
627,Bracket not showing as closed in Python 3<p>I'...,<p>You are using square brackets inside the fu...
424,Output shape of Sequential Network is wrong in...,"<p>With the current code, it is the expected o..."


The function in the cell below will query our tuned model using the `evaluation.input_text` and store the ground truth in `evaluation.output_text` in a DataFrame next to the model answers:

In [40]:
def create_eval_data(model, evaluation):
    model_answers = []

    for prompt in evaluation.input_text:
        response = model.generate_content(prompt)
        model_answers.append(response.text)

    eval_df = pd.DataFrame(
        {"candidate": model_answers, "reference": evaluation.output_text}
    )
    mask = eval_df.candidate == ""
    return eval_df[~mask]

In [41]:
eval_df = create_eval_data(tuned_model, evaluation)

In [43]:
eval_df.head()

,candidate,reference
771,"<p>This does it</p>\n<pre class=""lang-py prett...","<pre><code>&gt;&gt;&gt; col_names = ['Code', '..."
56,"<p>For anyone else stuck in this issue, it was...","<p>After speaking with Microsoft Support, it t..."
440,<p>I think there are some points to focus on:<...,<p>For this query:</p>\n<pre><code>SELECT scri...
627,<p>Your <code>eval_express</code> function tak...,<p>You are using square brackets inside the fu...
424,<p>You need to reshape your input and labels. ...,"<p>With the current code, it is the expected o..."


The function in the next cell computes the uni-gram BLEU and ROUGE scores. It averages these scores over all the reference answers and those generated by our tuned model, giving scores that can serve as performance metrics for our model.

In [44]:
def compute_scores(eval_data):
    predictions = eval_data.candidate.tolist()
    references = eval_data.reference.tolist()
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    rouge_value = rouge.compute(predictions=predictions, references=references)[
        "rouge1"
    ]
    bleu_value = bleu.compute(predictions=predictions, references=references)[
        "bleu"
    ]
    return {"rouge": rouge_value, "bleu": bleu_value}

In [45]:
compute_scores(eval_df)

{'rouge': 0.31049716525058924, 'bleu': 0.2078256786858426}

Given two versions of the model (possibly tuned with a different amount of data or training steps), you can now compare the scores to decide which one is the best. However, remember that these automated metrics are very crude proxy of human assessment. 

## Automated Evaluation


Let us conclude by noting that a Vertex tuning job collects and reports model tuning and model evaluation metrics, which can then be visualized in Vertex AI Studio by clicking on your tuned model name in the tuning section of Vertex AI Studio.
Here is a description of the metrics that are computed:


#### Model tuning metrics

The model tuning job automatically collects the following tuning metrics for `gemini-1.0-pro-002`.

* `/train_total_loss`: Loss for the tuning dataset at a training step.
* `/train_fraction_of_correct_next_step_preds`: The token accuracy at a training step. A single prediction consists of a sequence of tokens. This metric measures the accuracy of the predicted tokens when compared to the ground truth in the tuning dataset.
* `/train_num_predictions`: Number of predicted tokens at a training step.


#### Model validation metrics:

You can configure a model tuning job to collect the following validation metrics for gemini-1.0-pro-002 by passing an evaluation dataset as we did in this lab.

`/eval_total_loss`: Loss for the validation dataset at a validation step.
`/eval_fraction_of_correct_next_step_preds`: The token accuracy at an validation step. A single prediction consists of a sequence of tokens. This metric measures the accuracy of the predicted tokens when compared to the ground truth in the validation dataset.
`/eval_num_predictions`: Number of predicted tokens at a validation step.


The metrics visualizations are available after the model tuning job completes. 
If you don't specify a validation dataset when you create the tuning job, 
only the visualizations for the tuning metrics are available.



## Acknowledgement 

This notebook is adapted from a [tutorial](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/tuning/getting_started_tuning.ipynb)
written by Polong Lin.

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.